<a href="https://colab.research.google.com/github/cchang-vassar/Semantic-Relations-in-Vector-Embeddings/blob/main/study2_5_%5Bada003%5D_counterev_autoencoder_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ada-003 + EMNLP] Autoencoder: Generate Corresponding Embedding

## Set Up

### Imports

In [ ]:
!pip install tensorflow

In [ ]:
import os
import subprocess
import zipfile
import shutil
import pickle
import tensorflow as tf
import pandas as pd
import numpy as np
from google.colab import userdata
from scipy import spatial
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from plotnine import ggplot, geom_line, aes, ggsave, labs, theme, element_text, guides, guide_legend

### OSF Setup

In [ ]:
!pip install osfclient

In [ ]:
import osfclient.cli

In [ ]:
from osfclient.api import OSF
from osfclient.models import Project, Storage
from io import BytesIO

In [ ]:
os.environ["OSF_USERNAME"] = userdata.get("OSF_USERNAME")
OSF_USERNAME = os.environ["OSF_USERNAME"]

In [ ]:
os.environ["OSF_PASSWORD"] = userdata.get("OSF_PASSWORD")
OSF_PASSWORD = os.environ["OSF_PASSWORD"]

In [ ]:
os.environ["OSF_TOKEN"] = userdata.get("OSF_TOKEN")
OSF_TOKEN = os.environ["OSF_TOKEN"]

In [ ]:
os.environ["OSF_PROJECT_ID"] = userdata.get("OSF_PROJECT_ID")
OSF_PROJECT_ID = os.environ["OSF_PROJECT_ID"]

## Data

### Import corpora data from OSF

In [ ]:
!osf -p sakjg fetch osfstorage/data-dump/ada003-autoencoder/ada_evidence_embeddings_dump.zip

100% 35.3M/35.3M [00:00<00:00, 78.7Mbytes/s]


In [ ]:
embeddings_file_path = 'ada_evidence_embeddings_dump.zip'
output_folder_path = 'current-data-dump/embeddings-dump'
os.makedirs(output_folder_path, exist_ok=True)

with zipfile.ZipFile(embeddings_file_path, 'r') as zip_ref:
  zip_ref.extractall(output_folder_path)

extracted_files = os.listdir(output_folder_path)
print("Files extracted:", extracted_files)

Files extracted: ['argument_embeddings.pkl', 'evidence_embeddings.pkl', 'bad_argument_embeddings.pkl', 'bad_evidence_embeddings.pkl', '.ipynb_checkpoints']


In [ ]:
x = pd.read_pickle('current-data-dump/embeddings-dump/argument_embeddings.pkl')
y = pd.read_pickle('current-data-dump/embeddings-dump/evidence_embeddings.pkl')
x_train = x.iloc[:1499]
x_test = x.iloc[1499:]
y_train = y.iloc[:1499]
y_test = y.iloc[1499:]
combined_train_df = pd.concat([x_train, y_train], axis=1)
combined_test_df = pd.concat([x_test, y_test], axis=1)

combined_train_df = combined_train_df.groupby('argument').sample(1)
combined_test_df = combined_test_df.groupby('argument').sample(1)
x_train = combined_train_df.iloc[:, 0:1536].reset_index(drop=True)
y_train = combined_train_df.iloc[:, 1537:3073].reset_index(drop=True)
x_test = combined_test_df.iloc[:, 0:1536].reset_index(drop=True)
y_test = combined_test_df.iloc[:, 1537:3073].reset_index(drop=True)

### Make global data shuffled

In [ ]:
y_train_shuffled = y_train.copy().sample(frac=1).reset_index(drop=True)

### Save global training df

In [ ]:
global_x_train_folder_path = 'current-data-dump/ada003-autoencoder/'
global_x_train_file_path = f'{global_x_train_folder_path}global_x_train.pkl'
global_y_train_folder_path = 'current-data-dump/ada003-autoencoder/'
global_y_train_file_path = f'{global_x_train_folder_path}global_y_train.pkl'
os.makedirs(global_x_train_folder_path, exist_ok=True)
os.makedirs(global_y_train_folder_path, exist_ok=True)
with open(global_x_train_file_path, 'wb') as file:
  pickle.dump(x_train, file)
  print(f"File uploaded to {global_x_train_file_path}")
with open(global_y_train_file_path, 'wb') as file:
  pickle.dump(y_train, file)
  print(f"File uploaded to {global_y_train_file_path}")

File uploaded to current-data-dump/ada003-autoencoder/global_x_train.pkl
File uploaded to current-data-dump/ada003-autoencoder/global_y_train.pkl


In [ ]:
global_x_test_folder_path = 'current-data-dump/ada003-autoencoder/'
global_x_test_file_path = f'{global_x_test_folder_path}global_x_test.pkl'
global_y_test_folder_path = 'current-data-dump/ada003-autoencoder/'
global_y_test_file_path = f'{global_x_test_folder_path}global_y_test.pkl'
os.makedirs(global_x_test_folder_path, exist_ok=True)
os.makedirs(global_y_test_folder_path, exist_ok=True)
with open(global_x_test_file_path, 'wb') as file:
  pickle.dump(x_test, file)
  print(f"File uploaded to {global_x_test_file_path}")
with open(global_y_test_file_path, 'wb') as file:
  pickle.dump(y_test, file)
  print(f"File uploaded to {global_y_test_file_path}")

File uploaded to current-data-dump/ada003-autoencoder/global_x_test.pkl
File uploaded to current-data-dump/ada003-autoencoder/global_y_test.pkl


## Model

### Architecture

In [ ]:
# Layers
input_layer = tf.keras.layers.Input(shape=(1536, ), name="Input")
hidden_layer = tf.keras.layers.Dense(units=1536, activation="relu", name="Hidden")(input_layer)
output_layer = tf.keras.layers.Dense(units=1536, activation="linear", name="Output")(hidden_layer)

In [ ]:
# Model
autoencoder_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
autoencoder_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 1536)]            0         
                                                                 
 Hidden (Dense)              (None, 1536)              2360832   
                                                                 
 Output (Dense)              (None, 1536)              2360832   
                                                                 
Total params: 4721664 (18.01 MB)
Trainable params: 4721664 (18.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Metric

In [ ]:
@tf.keras.saving.register_keras_serializable()
def metric_choose_argument_global_y_train(y_true, y_pred):
  """global_metric"""
  global_training_df_32 = tf.cast(x_train, dtype=tf.float32)

  cos_sim_pred = tf.matmul(global_training_df_32, y_pred, transpose_b=True) / tf.reshape(tf.norm(y_pred) * tf.norm(global_training_df_32, axis=1), [-1, 1])
  cos_sim_true = tf.matmul(global_training_df_32, y_true, transpose_b=True) / tf.reshape(tf.norm(y_true) * tf.norm(global_training_df_32, axis=1), [-1, 1])

  max_cos_sim_pred = tf.math.argmax(cos_sim_pred)
  max_cos_sim_true = tf.math.argmax(cos_sim_true)

  return tf.math.count_nonzero(tf.equal(max_cos_sim_pred, max_cos_sim_true))

### Global Training

In [ ]:
# Global Model
global_autoencoder_model = tf.keras.models.clone_model(autoencoder_model)
global_autoencoder_model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
  loss="cosine_similarity",
  metrics=[metric_choose_argument_global_y_train]
)

In [ ]:
checkpoint_callback = ModelCheckpoint(filepath='global_autoencoder_weights.keras', save_best_only=False, save_weights_only=False, verbose=1)
csv_logger_callback = CSVLogger(filename='global_training_log.csv', separator=',', append=True)
global_history = global_autoencoder_model.fit(
  x=x_train,
  y=y_train,
  batch_size=1,
  epochs=100,
  validation_data = (x_test, y_test),
  callbacks=[checkpoint_callback, csv_logger_callback]
)

Epoch 1/100
59/71 [=======================>......] - ETA: 0s - loss: -0.3018 - metric_choose_argument_global_y_train: 0.0169
Epoch 1: saving model to global_autoencoder_weights.keras
71/71 [==============================] - 5s 11ms/step - loss: -0.3118 - metric_choose_argument_global_y_train: 0.0141 - val_loss: -0.3520 - val_metric_choose_argument_global_y_train: 0.0625
Epoch 2/100
69/71 [============================>.] - ETA: 0s - loss: -0.4344 - metric_choose_argument_global_y_train: 0.1159
Epoch 2: saving model to global_autoencoder_weights.keras
71/71 [==============================] - 1s 8ms/step - loss: -0.4370 - metric_choose_argument_global_y_train: 0.1268 - val_loss: -0.3552 - val_metric_choose_argument_global_y_train: 0.0625
Epoch 3/100
61/71 [========================>.....] - ETA: 0s - loss: -0.5748 - metric_choose_argument_global_y_train: 0.5082
Epoch 3: saving model to global_autoencoder_weights.keras
71/71 [==============================] - 1s 15ms/step - loss: -0.5712 - 

In [ ]:
global_history_df = pd.DataFrame(global_history.history)

In [ ]:
output_folder_path = 'current-data-dump/ada003-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
global_history_df.to_csv(f'{output_folder_path}global_training_log.csv')

In [ ]:
output_folder_path = 'current-data-dump/ada003-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
global_autoencoder_model.save(f'{output_folder_path}global_autoencoder_model.keras')

### Global Shuffled Training

In [ ]:
# Global Shuffled Model
global_shuffled_autoencoder_model = tf.keras.models.clone_model(autoencoder_model)
global_shuffled_autoencoder_model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
  loss="cosine_similarity",
  metrics=[metric_choose_argument_global_y_train]
)

global_shuffled_history = global_shuffled_autoencoder_model.fit(
  x=x_train,
  y=y_train_shuffled,
  batch_size=1,
  epochs=20,
  validation_data = (x_test, y_test)
)

Epoch 1/20
71/71 [==============================] - 4s 50ms/step - loss: -0.3090 - metric_choose_argument_global_y_train: 0.0000e+00 - val_loss: -0.3476 - val_metric_choose_argument_global_y_train: 0.0625
Epoch 2/20
71/71 [==============================] - 3s 46ms/step - loss: -0.3884 - metric_choose_argument_global_y_train: 0.0141 - val_loss: -0.3355 - val_metric_choose_argument_global_y_train: 0.0625
Epoch 3/20
71/71 [==============================] - 4s 53ms/step - loss: -0.4668 - metric_choose_argument_global_y_train: 0.1549 - val_loss: -0.2930 - val_metric_choose_argument_global_y_train: 0.0000e+00
Epoch 4/20
71/71 [==============================] - 3s 49ms/step - loss: -0.6007 - metric_choose_argument_global_y_train: 0.3521 - val_loss: -0.2738 - val_metric_choose_argument_global_y_train: 0.0000e+00
Epoch 5/20
71/71 [==============================] - 3s 45ms/step - loss: -0.7384 - metric_choose_argument_global_y_train: 0.6761 - val_loss: -0.2325 - val_metric_choose_argument_global

In [ ]:
global_shuffled_history_df = pd.DataFrame(global_shuffled_history.history)

In [ ]:
output_folder_path = 'current-data-dump/ada003-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
global_shuffled_history_df.to_csv(f'{output_folder_path}global_shuffled_training_log.csv')

In [ ]:
output_folder_path = 'current-data-dump/ada003-autoencoder/'
os.makedirs(output_folder_path, exist_ok=True)
global_shuffled_autoencoder_model.save(f'{output_folder_path}global_shuffled_autoencoder_model.keras')

## Export data

In [ ]:
def export_ada_autoencoder():
  """Export ada_autoencoder directory"""
  ada_autoencoder_file_path = 'current-data-dump/ada003-autoencoder'
  ada_autoencoder_file_path_zip = 'current-data-dump/ada003-autoencoder'
  shutil.make_archive(ada_autoencoder_file_path_zip, 'zip', ada_autoencoder_file_path)
  print(f"Zip file created at: {ada_autoencoder_file_path_zip}")
  result = subprocess.run([f"osf -p sakjg upload --force {ada_autoencoder_file_path_zip}.zip data-dump/ada003-evidence-autoencoder/ada003_evidence_autoencoder.zip"], shell=True, capture_output=True, text=True)
  print(result.stderr)
  print(f"File: {ada_autoencoder_file_path_zip} uploaded at osfstorage")

In [ ]:
export_ada_autoencoder()

Zip file created at: current-data-dump/ada003-autoencoder

File: current-data-dump/ada003-autoencoder uploaded at osfstorage


## Import data

In [ ]:
def import_ada_autoencoder():
  """Import ada_autoencoder directory"""
  subprocess.run("osf -p sakjg fetch --force osfstorage/data-dump/ada003-evidence-autoencoder/ada003_evidence_autoencoder.zip", shell=True)
  print("ada_autoencoder.zip successfully imported")
  ada_autoencoder_file_path_zip = 'ada003_evidence_autoencoder.zip'
  ada_autoencoder_file_path = 'current-data-dump/ada003-autoencoder'
  os.makedirs(ada_autoencoder_file_path, exist_ok=True)
  with zipfile.ZipFile(ada_autoencoder_file_path_zip, 'r') as zip_ref:
    zip_ref.extractall(ada_autoencoder_file_path)
  extracted_files = os.listdir(ada_autoencoder_file_path)
  print("Files extracted:", extracted_files)

## Load global training history

In [ ]:
import_ada_autoencoder()

ada_autoencoder.zip successfully imported
Files extracted: ['global_x_train.pkl', 'global_y_train.pkl', 'global_training_log.csv', 'global_shuffled_training_log.csv', 'global_autoencoder_model.keras', 'global_shuffled_autoencoder_model.keras', 'global_x_test.pkl', 'global_y_test.pkl']


#### Unshuffled training history

In [ ]:
# Access training history
loaded_global_history = pd.DataFrame(pd.read_csv("current-data-dump/ada003-autoencoder/global_training_log.csv"))
loaded_global_history = pd.melt(loaded_global_history, id_vars='Unnamed: 0', value_vars=['metric_choose_argument_global_y_train', 'val_metric_choose_argument_global_y_train'], var_name='dataset', value_name='accuracy')
loaded_global_history = loaded_global_history.replace(['metric_choose_argument_global_y_train', 'val_metric_choose_argument_global_y_train'], ['training set', 'validation set'])
loaded_global_history.rename(columns = {'Unnamed: 0':'epoch'}, inplace = True)
loaded_global_history['shuffled'] = False

In [ ]:
global_training_plot = ggplot(loaded_global_history, aes(x='epoch', y='accuracy', linetype='dataset')) + geom_line() + labs(title='Learning Curve of Model Trained on Unshuffled Data', x='Epoch', y='Accuracy')
ggsave(global_training_plot, "current-data-dump/ada003-autoencoder/global_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: current-data-dump/ada002-autoencoder/global_training_plot.png


#### Shuffled training history

In [ ]:
# Access training history
loaded_global_shuffled_history = pd.DataFrame(pd.read_csv("current-data-dump/ada003-autoencoder/global_shuffled_training_log.csv"))
loaded_global_shuffled_history = pd.melt(loaded_global_shuffled_history, id_vars='Unnamed: 0', value_vars=['metric_choose_argument_global_y_train', 'val_metric_choose_argument_global_y_train'], var_name='dataset', value_name='accuracy')
loaded_global_shuffled_history = loaded_global_shuffled_history.replace(['metric_choose_argument_global_y_train', 'val_metric_choose_argument_global_y_train'], ['training set', 'validation set'])
loaded_global_shuffled_history.rename(columns = {'Unnamed: 0':'epoch'}, inplace = True)
loaded_global_shuffled_history['shuffled'] = False

In [ ]:
global_shuffled_training_plot = ggplot(loaded_global_shuffled_history, aes(x='epoch', y='accuracy', linetype='dataset')) + geom_line() + labs(title='Learning Curve of Model Trained on Shuffled Data', x='Epoch', y='Accuracy')
ggsave(global_shuffled_training_plot, "current-data-dump/ada003-autoencoder/global_shuffled_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 6.4 x 4.8 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: current-data-dump/ada002-autoencoder/global_shuffled_training_plot.png


## Combined Training Plots

In [ ]:
combined_global_training_df = pd.concat([loaded_global_history, loaded_global_shuffled_history])

In [ ]:
combined_global_plot = (
  ggplot(combined_global_training_df, aes(x='epoch', y='accuracy', linetype='dataset', color='shuffled')) +
  geom_line(size=2) +
  labs(title='Learning Curve of Model Trained on Unshuffled vs. Within-Topic Shuffled Data', x='Epoch', y='Accuracy') +
  theme(
    figure_size=(16,24),
    axis_title=element_text(size=32),
    axis_text=element_text(size=24),
    legend_title=element_text(size=32, lineheight=1.5),
    legend_text=element_text(size=24, lineheight=1.5),
    plot_title=element_text(size=40, wrap=True, lineheight=1.5),
    legend_position="bottom",
    legend_key_width=64
  ) +
  guides(fill = guide_legend(byrow = True))
)
ggsave(combined_global_plot, "current-data-dump/ada003-autoencoder/combined_global_training_plot.png")

/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:587: PlotnineWarning: Saving 16 x 24 in image.
/usr/local/lib/python3.10/dist-packages/plotnine/ggplot.py:588: PlotnineWarning: Filename: current-data-dump/ada002-autoencoder/combined_global_training_plot.png


## Final Export

In [ ]:
export_ada_autoencoder()

Zip file created at: current-data-dump/ada-autoencoder

File: current-data-dump/ada-autoencoder uploaded at osfstorage
